# Feature Engineering 

- Aditya Rachman Putra -23520032

- Harits Abdurrohman - 23520047

## GLCM (Gray level co-occurrence Matrix)

## Color Moments (4 moments)

## Kombinasi (GLCM dan Color Moment)

In [1]:
from pycocotools.coco import COCO
from sklearn import tree, metrics
from skimage.transform import rescale, resize, downscale_local_mean
from skimage.util import crop
from skimage import io, color
from math import floor, ceil
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from sklearn.decomposition import TruncatedSVD
import json
from LoadLocalCOCO import LoadLocalCOCO as llc
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import pickle
from joblib import dump, load
from tqdm import tqdm
import cv2
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, PredefinedSplit
import xgboost as xgb

# GLCM
from skimage.feature import greycomatrix, greycoprops

%matplotlib inline


In [2]:
def flatten_list(qlist):
    return [y for x in qlist for y in x]

In [3]:
train_instance="annotations/instances_train2017.json"
val_instance="annotations/instances_val2017.json"

coco_path = "D:/FSR/COCO/train2017"

classes = ['dog', 'cat']

coco_train = llc(train_instance, coco_path)
coco_val = llc(val_instance, coco_path)

loading annotations into memory...
Done (t=28.41s)
creating index...
index created!
loading annotations into memory...
Done (t=0.76s)
creating index...
index created!


In [4]:
data_train = []
y_train = []
for i, category in enumerate(classes):
    data = coco_train.get_cropped_images_flatten_and_resized(category, resize=(64, 64), flatten=False)
    data_train.append(data)
    y_train += np.array(len(data)) *[abs(1-i)]

5508it [05:30, 16.69it/s]
2it [00:00, 17.62it/s]Broken images: [7125, 16775, 24247, 24247, 55296, 47386, 69468, 35613, 77325, 12696, 22427, 26321, 54957, 790, 1360, 55478, 42341, 57631, 2400, 10276, 94049, 58741, 5782, 44613, 58741, 24159, 44613, 29176] images
4768it [04:42, 16.91it/s]Broken images: [5617, 711, 39171, 9413, 46749, 18211, 26375, 26375, 26375, 66041] images



In [ ]:
data_val = []
y_val = []
for i, category in enumerate(classes):
    data = coco_val.get_cropped_images_flatten_and_resized(category, resize=(64, 64), flatten=False)
    data_train.append(data)
    y_label += np.array(len(data)) *[abs(1-i)]

In [5]:
np.shape(data_train[0])

(5480, 64, 64, 3)

In [11]:
y_train = np.array(len(data_train[0]) *[1] + len(data_train[1]) *[0])
np.shape(y_train)

(10238,)

In [6]:
data_train_flat = flatten_list(data_train)

In [7]:
np.shape(data_train_flat)

(10238, 64, 64, 3)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(data_train_flat, y_train, test_size=0.3, random_state=101)

In [23]:
print('data train dimension:', np.shape(X_train))
print('label train dimension:', np.shape(y_train))
print('data test dimension:', np.shape(X_test))
print('label test dimension:', np.shape(y_test))

data train dimension: (7166, 64, 64, 3)
label train dimension: (7166,)
data test dimension: (3072, 64, 64, 3)
label test dimension: (3072,)


In [19]:
# tekstur
# glcm
def glcm_feature_extraction(image_data, distance=[5], angles=[0], levels=256, symmetric=True, normed=True, features=['contrast', 'dissimilarity', 'homogeneity', 'ASM', 'energy', 'correlation']):
    feature_data = []
    for image in image_data:
        patch_feature = {}
        if len(image.shape)==3:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        glcm = greycomatrix(image, distances=distance, angles=angles, levels=levels, symmetric=symmetric, normed=normed)
        for feature in features:
            patch_feature[feature] = greycoprops(glcm, feature)[0, 0]
        feature_data.append(patch_feature)
    feature_data = pd.DataFrame(feature_data)
    return feature_data

In [20]:
glcm_data_train = glcm_feature_extraction(X_train)

In [24]:
np.shape(glcm_data_train)

(7166, 6)

In [25]:
glcm_data_train.head()

,contrast,dissimilarity,homogeneity,ASM,energy,correlation
0,3420.117585,38.985169,0.180919,0.020414,0.142879,0.469978
1,1924.596928,31.621292,0.043353,0.000183,0.013513,0.672204
2,2398.074682,37.066737,0.036011,0.000185,0.013612,0.581640
3,2249.118114,33.344280,0.045304,0.000217,0.014744,0.457202
4,3398.435117,43.893803,0.037141,0.000173,0.013139,0.466568


In [26]:
glcm_data_train.to_csv('glcm_data_train.csv')

In [27]:
glcm_data_test = glcm_feature_extraction(X_test)
glcm_data_test.head()

,contrast,dissimilarity,homogeneity,ASM,energy,correlation
0,3773.738877,37.036017,0.059179,0.000343,0.018514,0.512819
1,4234.574153,51.815678,0.018144,0.000163,0.012770,0.245960
2,2978.331568,35.094280,0.076492,0.000277,0.016652,0.549114
3,1631.094280,26.395657,0.079384,0.000433,0.020813,0.635220
4,2421.687235,29.733845,0.140922,0.001294,0.035967,0.807161


In [28]:
np.shape(glcm_data_test)

(3072, 6)

In [29]:
glcm_data_test.to_csv('glcm_data_test.csv')

In [36]:
# color moment
# moment 1: mean
# moment 2: standard deviation
# moment 3: skewness
# moment 4: kurtosis

def modified_skew(image):
    mean = np.average(image)
    dime = np.shape(image)[0]*np.shape(image)[1]
    skew_img = np.power((image.astype(np.int8) - mean), 3).sum()
    skew_img *= 1/dime
    skew_img = np.cbrt(skew_img)
    return skew_img

def modified_skew2(image):
    mean = np.average(image)
    dime = np.shape(image)[0]*np.shape(image)[1]
    std = np.std(image)
    skew_img = np.power(((image.astype(np.int8) - mean)/std), 3)
    skew_img *= 1/dime
    return skew_img

def modified_kurtosis(image):
    mean = np.average(image)
    std = np.std(image)
    dime = np.shape(image)[0]*np.shape(image)[1]
    kurt = (np.power(((image.astype(np.int8) - mean)/std), 4)-3).sum()
    kurt *= 1/dime
    return kurt


def color_moment_extraction_3channel(image_data):
    feature_data = []
    for image in image_data:
        image_features = {}
        for ch in range(image.shape[2]):
            mean_img = np.average(image[:,:,ch])
            image_features[f'mean_{ch}'] = mean_img
            std_img = np.std(image[:,:,ch])
            image_features[f'std_{ch}'] = std_img
            skew_img = modified_skew(image[:,:,ch])
            image_features[f'skewness_{ch}'] = skew_img
            kurtosis_img = modified_kurtosis(image[:,:,ch])
            image_features[f'kurtosis_{ch}'] = kurtosis_img
        feature_data.append(image_features)
    feature_data = pd.DataFrame(feature_data)
    return feature_data

def color_moment_extraction_1channel(image_data):
    feature_data = []
    for image in image_data:
        image_features = {}
        mean_img = np.average(image)
        image_features['mean'] = mean_img
        std_img = np.std(image)
        image_features['std'] = std_img
        skew_img = modified_skew(image)
        image_features['skewness'] = skew_img
        kurtosis_img = modified_kurtosis(image)
        image_features['kurtosis'] = kurtosis_img
        feature_data.append(image_features)
    feature_data = pd.DataFrame(feature_data)
    return feature_data


In [37]:
color_moment_data_train = color_moment_extraction_3channel(X_train)

In [38]:
np.shape(color_moment_data_train)

(7166, 12)

In [39]:
color_moment_data_train.head()

,mean_0,std_0,skewness_0,kurtosis_0,mean_1,std_1,skewness_1,kurtosis_1,mean_2,std_2,skewness_2,kurtosis_2
0,40.283691,47.217526,-51.452960,1.824191,50.716553,54.542173,-69.332647,2.886897,59.271729,66.280396,-78.626445,0.815929
1,117.266113,54.028064,-169.618781,119.385076,137.581299,53.966440,-182.708996,159.307759,150.463623,61.453811,-194.050390,116.677815
2,102.685059,46.271431,-140.534325,121.591130,114.159180,53.148433,-158.986903,102.063759,131.738770,61.683033,-172.325381,72.075658
3,100.934082,43.009731,-151.855815,214.096365,95.221436,43.440451,-143.086858,168.094073,105.747070,49.894992,-160.805095,136.802244
4,81.591797,58.973444,-113.327495,17.895513,143.205566,61.120165,-177.192808,86.709248,194.368652,53.827913,-233.429790,386.808901


In [40]:
color_moment_data_train.to_csv('color_moment_data_train.csv')

In [42]:
color_moment_data_test = color_moment_extraction_3channel(X_test)

In [43]:
np.shape(color_moment_data_test)

(3072, 12)

In [44]:
color_moment_data_test.head()

,mean_0,std_0,skewness_0,kurtosis_0,mean_1,std_1,skewness_1,kurtosis_1,mean_2,std_2,skewness_2,kurtosis_2
0,60.152100,65.079407,-62.010704,-1.183716,69.447021,66.074385,-71.241946,-0.143536,87.151123,64.355157,-91.016590,4.596184
1,80.335449,46.997063,-99.113339,33.314412,113.894043,55.695610,-156.984285,81.010740,127.169678,54.839149,-174.157767,125.894199
2,115.224365,56.153656,-130.018200,42.320266,116.757324,57.341133,-125.358601,33.340832,115.312012,61.742897,-126.016276,23.901065
3,42.412354,37.417019,-18.033385,0.856238,60.638672,47.685312,-81.618153,17.466625,68.400146,52.706224,-114.183328,31.767681
4,72.557617,73.811422,-95.680237,1.480019,89.187500,78.396413,-119.348424,4.649973,112.766846,86.220503,-143.934361,6.840237


In [45]:
color_moment_data_test.to_csv('color_moment_data_test.csv')

In [43]:
# texture
# HOG
from skimage import feature
from sklearn.decomposition import PCA, IncrementalPCA
from scipy import sparse
def hog_feature_extraction(image_data, orientations=9, pixels_per_cell=(8, 8),
	cells_per_block=(2, 2), transform_sqrt=True, block_norm="L1", n_components=10, threshold=0.45):
    feature_data = []
    hog_features = []
    # pca = PCA(n_components = n_components, svd_solver='full')
    pca = IncrementalPCA(n_components=n_components, batch_size=300)
    for image in image_data:
        hog_f = feature.hog(image, orientations=orientations, pixels_per_cell=pixels_per_cell,
	cells_per_block=cells_per_block, transform_sqrt=transform_sqrt, block_norm=block_norm)
        hog_features.append(hog_f)
    hog_features = np.array(hog_features)
    hog_features[hog_features==np.inf] = 0
    hog_features[hog_features==np.nan] = 0
    print(hog_features.shape)
    # print(np.isnan(np.sum(hog_features)))
    pca.partial_fit(hog_features[:100, :])
    hog_features_sparse = sparse.csr_matrix(hog_features)
    # pca.fit(hog_features_sparse)
    reduced_data_pca = pca.fit_transform(hog_features_sparse)
    for image_pc in reduced_data_pca:
        selected_pc = []
        pc_sum = 0
        for pc in image_pc:
            if pc_sum <=threshold:
                pc_sum += pc
                selected_pc.append(pc)
        features_tmp = {}
        for i, pc in enumerate(selected_pc):
            features_tmp[f'pc_{i}'] = pc
        feature_data.append(features_tmp)    
    feature_data = pd.DataFrame(feature_data)       
    return feature_data

In [129]:
## Bagian GridSearch
# Eksperimen dengan GLCM
test_fold = np.concatenate([
    # The training data.
    np.full(len(glcm_data_train), -1, dtype=np.int8),
    # The development data.
    np.zeros(len(glcm_data_test), dtype=np.int8)
])

cv = PredefinedSplit(test_fold)

X = np.concatenate([glcm_data_train, glcm_data_test])
y = np.concatenate([y_train, y_test])

In [130]:
base = DecisionTreeClassifier()
grid = {
"max_depth" : [10, 100, 1000],
"criterion" : ["gini", "entropy"]
}
model = GridSearchCV(base, grid, cv=cv,n_jobs=4, verbose=100, return_train_score = True)

In [131]:
model = model.fit(X,y)

Fitting 1 folds for each of 6 candidates, totalling 6 fits
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
Pickling array (shape=(10238, 6), dtype=float64).
Pickling array (shape=(10238,), dtype=int32).
Pickling array (shape=(7166,), dtype=int32).
Pickling array (shape=(3072,), dtype=int32).
Pickling array (shape=(10238, 6), dtype=float64).
Pickling array (shape=(10238,), dtype=int32).
Pickling array (shape=(7166,), dtype=int32).
Pickling array (shape=(3072,), dtype=int32).
Pickling array (shape=(10238, 6), dtype=float64).
Pickling array (shape=(10238,), dtype=int32).
Pickling array (shape=(7166,), dtype=int32).
Pickling array (shape=(3072,), dtype=int32).
Pickling array (shape=(10238, 6), dtype=float64).
Pickling array (shape=(10238,), dtype=int32).
Pickling array (shape=(7166,), dtype=int32).
Pickling array (shape=(3072,), dtype=int32).
Pickling array (shape=(10238, 6), dtype=float64).
Pickling array (shape=(10238,), dtype=int32).
Pickling array (shape=(716

In [132]:
print(model.cv_results_)

{'mean_fit_time': array([0.04999995, 0.07399964, 0.07499647, 0.08149743, 0.18499851,
       0.15199995]), 'std_fit_time': array([0., 0., 0., 0., 0., 0.]), 'mean_score_time': array([0.00100064, 0.00099945, 0.00149989, 0.00050187, 0.00099969,
       0.00099969]), 'std_score_time': array([0., 0., 0., 0., 0., 0.]), 'param_criterion': masked_array(data=['gini', 'gini', 'gini', 'entropy', 'entropy',
                   'entropy'],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_max_depth': masked_array(data=[10, 100, 1000, 10, 100, 1000],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'criterion': 'gini', 'max_depth': 10}, {'criterion': 'gini', 'max_depth': 100}, {'criterion': 'gini', 'max_depth': 1000}, {'criterion': 'entropy', 'max_depth': 10}, {'criterion': 'entropy', 'max_depth': 100}, {'criterion': 'entropy', 'max_depth': 1000}], 'split0_test

In [133]:
pd.concat([pd.DataFrame(model.cv_results_["rank_test_score"], columns=["Rank"]), pd.DataFrame(model.cv_results_["params"]),pd.DataFrame(model.cv_results_["mean_test_score"], columns=["Accuracy"]),pd.DataFrame(model.cv_results_["mean_fit_time"], columns=["Fit Time"])],axis=1)

,Rank,criterion,max_depth,Accuracy,Fit Time
0,2,gini,10,0.554362,0.050000
1,3,gini,100,0.531250,0.074000
2,4,gini,1000,0.526693,0.074996
3,1,entropy,10,0.561523,0.081497
4,6,entropy,100,0.510417,0.184999
5,5,entropy,1000,0.513672,0.152000


In [134]:
pd.concat([pd.DataFrame(model.cv_results_["rank_test_score"], columns=["Rank"]), pd.DataFrame(model.cv_results_["params"]),pd.DataFrame(model.cv_results_["mean_test_score"], columns=["Accuracy"]),pd.DataFrame(model.cv_results_["mean_train_score"], columns=["Train Score"])],axis=1)

,Rank,criterion,max_depth,Accuracy,Train Score
0,2,gini,10,0.554362,0.670109
1,3,gini,100,0.531250,1.000000
2,4,gini,1000,0.526693,1.000000
3,1,entropy,10,0.561523,0.636478
4,6,entropy,100,0.510417,1.000000
5,5,entropy,1000,0.513672,1.000000


In [135]:
dump(model, 'local_dtl_{}_{}_glcm.pkl'.format('70p', 64))

['local_dtl_70p_64_glcm.pkl']

In [136]:
# Random Forest classifier
rf_base = xgb.XGBRFClassifier()
rf_grid = {
"max_depth" : [4, 6, 10],
"learning_rate" : [0.1, 0.3],
"n_estimators" : [1, 5, 10, 50]
}
rf_model = GridSearchCV(rf_base, rf_grid, cv=cv,n_jobs=4, verbose=100, return_train_score = True)

In [137]:
rf_model = rf_model.fit(X, y)

Fitting 1 folds for each of 24 candidates, totalling 24 fits
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
Pickling array (shape=(10238, 6), dtype=float64).
Pickling array (shape=(10238,), dtype=int32).
Pickling array (shape=(7166,), dtype=int32).
Pickling array (shape=(3072,), dtype=int32).
Pickling array (shape=(10238, 6), dtype=float64).
Pickling array (shape=(10238,), dtype=int32).
Pickling array (shape=(7166,), dtype=int32).
Pickling array (shape=(3072,), dtype=int32).
Pickling array (shape=(10238, 6), dtype=float64).
Pickling array (shape=(10238,), dtype=int32).
Pickling array (shape=(7166,), dtype=int32).
Pickling array (shape=(3072,), dtype=int32).
Pickling array (shape=(10238, 6), dtype=float64).
Pickling array (shape=(10238,), dtype=int32).
Pickling array (shape=(7166,), dtype=int32).
Pickling array (shape=(3072,), dtype=int32).
Pickling array (shape=(10238, 6), dtype=float64).
Pickling array (shape=(10238,), dtype=int32).
Pickling array (shape=(7

In [138]:
print(rf_model.cv_results_)

{'mean_fit_time': array([0.01099873, 0.02799964, 0.04100013, 0.16299868, 0.01500034,
       0.0384984 , 0.05799937, 0.24850035, 0.0199995 , 0.04900002,
       0.08999753, 0.40200114, 0.01700115, 0.02500105, 0.04400134,
       0.1709969 , 0.01649833, 0.03950024, 0.06049871, 0.23050237,
       0.02000165, 0.05199909, 0.08399987, 0.31301069]), 'std_fit_time': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.]), 'mean_score_time': array([0.00299883, 0.00300002, 0.00299954, 0.00400162, 0.00250053,
       0.00300002, 0.00349975, 0.00650024, 0.00249934, 0.00350046,
       0.00350094, 0.00899911, 0.0025003 , 0.00299931, 0.00249839,
       0.00449967, 0.00250149, 0.00300312, 0.00300145, 0.00599909,
       0.00250006, 0.00299931, 0.00349998, 0.00848889]), 'std_score_time': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.]), 'param_learning_rate': masked_array(data=[0.1, 0.1, 0.1

In [140]:
pd.concat([pd.DataFrame(rf_model.cv_results_["rank_test_score"], columns=["Rank"]), pd.DataFrame(rf_model.cv_results_["params"]),pd.DataFrame(rf_model.cv_results_["mean_test_score"], columns=["Accuracy"]),pd.DataFrame(rf_model.cv_results_["mean_fit_time"], columns=["Fit Time"])],axis=1)

,Rank,learning_rate,max_depth,n_estimators,Accuracy,Fit Time
0,21,0.1,4,1,0.559570,0.010999
1,17,0.1,4,5,0.565755,0.028000
2,3,0.1,4,10,0.574870,0.041000
3,1,0.1,4,50,0.580078,0.162999
4,13,0.1,6,1,0.566732,0.015000
5,11,0.1,6,5,0.567383,0.038498
6,7,0.1,6,10,0.568359,0.057999
7,5,0.1,6,50,0.572266,0.248500
8,23,0.1,10,1,0.554036,0.020000
9,13,0.1,10,5,0.566732,0.049000


In [141]:
pd.concat([pd.DataFrame(rf_model.cv_results_["rank_test_score"], columns=["Rank"]), pd.DataFrame(rf_model.cv_results_["params"]),pd.DataFrame(rf_model.cv_results_["mean_test_score"], columns=["Accuracy"]),pd.DataFrame(rf_model.cv_results_["mean_train_score"], columns=["Train Score"])],axis=1)

,Rank,learning_rate,max_depth,n_estimators,Accuracy,Train Score
0,21,0.1,4,1,0.559570,0.578845
1,17,0.1,4,5,0.565755,0.593358
2,3,0.1,4,10,0.574870,0.598660
3,1,0.1,4,50,0.580078,0.594474
4,13,0.1,6,1,0.566732,0.591404
5,11,0.1,6,5,0.567383,0.630059
6,7,0.1,6,10,0.568359,0.629221
7,5,0.1,6,50,0.572266,0.632850
8,23,0.1,10,1,0.554036,0.628384
9,13,0.1,10,5,0.566732,0.727463


In [142]:
dump(rf_model, 'local_rf_{}_{}_glcm.pkl'.format('70p', 64))

['local_rf_70p_64_glcm.pkl']

In [143]:
xg_base = xgb.XGBClassifier()
xg_grid = {
"max_depth" : [4, 6, 10],
"learning_rate" : [0.1, 0.3],
"n_estimators" : [1, 5, 10, 50]
}
xg_model = GridSearchCV(xg_base, xg_grid, cv=cv, n_jobs=4, verbose=100, return_train_score = True)

In [144]:
xg_model = xg_model.fit(X, y)

Fitting 1 folds for each of 24 candidates, totalling 24 fits
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
Pickling array (shape=(10238, 6), dtype=float64).
Pickling array (shape=(10238,), dtype=int32).
Pickling array (shape=(7166,), dtype=int32).
Pickling array (shape=(3072,), dtype=int32).
Pickling array (shape=(10238, 6), dtype=float64).
Pickling array (shape=(10238,), dtype=int32).
Pickling array (shape=(7166,), dtype=int32).
Pickling array (shape=(3072,), dtype=int32).
Pickling array (shape=(10238, 6), dtype=float64).
Pickling array (shape=(10238,), dtype=int32).
Pickling array (shape=(7166,), dtype=int32).
Pickling array (shape=(3072,), dtype=int32).
Pickling array (shape=(10238, 6), dtype=float64).
Pickling array (shape=(10238,), dtype=int32).
Pickling array (shape=(7166,), dtype=int32).
Pickling array (shape=(3072,), dtype=int32).
Pickling array (shape=(10238, 6), dtype=float64).
Pickling array (shape=(10238,), dtype=int32).
Pickling array (shape=(7

In [145]:
print(xg_model.cv_results_)

{'mean_fit_time': array([0.02900028, 0.0979991 , 0.06999755, 0.28349924, 0.0224998 ,
       0.06049514, 0.08199859, 0.32499909, 0.0199995 , 0.06749892,
       0.11599851, 0.46350026, 0.01549959, 0.03799272, 0.05301571,
       0.20699716, 0.01897931, 0.03950119, 0.07550097, 0.2785008 ,
       0.02100611, 0.06049967, 0.11200023, 0.33850265]), 'std_fit_time': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.]), 'mean_score_time': array([0.00399899, 0.00599957, 0.00400019, 0.00450134, 0.00450158,
       0.00350237, 0.00400019, 0.00550103, 0.00299978, 0.00350237,
       0.00350046, 0.00850058, 0.0025003 , 0.00300097, 0.0030005 ,
       0.00499988, 0.00350094, 0.00299835, 0.00301719, 0.00549817,
       0.00249243, 0.00300193, 0.00399899, 0.00702667]), 'std_score_time': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.]), 'param_learning_rate': masked_array(data=[0.1, 0.1, 0.1

In [147]:
pd.concat([pd.DataFrame(xg_model.cv_results_["rank_test_score"], columns=["Rank"]), pd.DataFrame(xg_model.cv_results_["params"]),pd.DataFrame(xg_model.cv_results_["mean_test_score"], columns=["Accuracy"]),pd.DataFrame(xg_model.cv_results_["mean_fit_time"], columns=["Fit Time"])],axis=1)

,Rank,learning_rate,max_depth,n_estimators,Accuracy,Fit Time
0,14,0.1,4,1,0.561198,0.029000
1,6,0.1,4,5,0.568034,0.097999
2,3,0.1,4,10,0.571940,0.069998
3,2,0.1,4,50,0.572917,0.283499
4,14,0.1,6,1,0.561198,0.022500
5,10,0.1,6,5,0.564128,0.060495
6,12,0.1,6,10,0.563151,0.081999
7,4,0.1,6,50,0.571289,0.324999
8,23,0.1,10,1,0.548828,0.020000
9,20,0.1,10,5,0.556966,0.067499


In [148]:
pd.concat([pd.DataFrame(xg_model.cv_results_["rank_test_score"], columns=["Rank"]), pd.DataFrame(xg_model.cv_results_["params"]),pd.DataFrame(xg_model.cv_results_["mean_test_score"], columns=["Accuracy"]),pd.DataFrame(xg_model.cv_results_["mean_train_score"], columns=["Train Score"])],axis=1)

,Rank,learning_rate,max_depth,n_estimators,Accuracy,Train Score
0,14,0.1,4,1,0.561198,0.589590
1,6,0.1,4,5,0.568034,0.602707
2,3,0.1,4,10,0.571940,0.609126
3,2,0.1,4,50,0.572917,0.631873
4,14,0.1,6,1,0.561198,0.609824
5,10,0.1,6,5,0.564128,0.635361
6,12,0.1,6,10,0.563151,0.646107
7,4,0.1,6,50,0.571289,0.700809
8,23,0.1,10,1,0.548828,0.681970
9,20,0.1,10,5,0.556966,0.741976


In [149]:
dump(xg_model, 'local_xg_{}_{}_glcm.pkl'.format('70p', 64))

['local_xg_70p_64_glcm.pkl']

In [150]:
# Eksperimen dengan Color moment
test_fold = np.concatenate([
    # The training data.
    np.full(len(color_moment_data_train), -1, dtype=np.int8),
    # The development data.
    np.zeros(len(color_moment_data_test), dtype=np.int8)
])

cv = PredefinedSplit(test_fold)

X = np.concatenate([color_moment_data_train, color_moment_data_test])
y = np.concatenate([y_train, y_test])

In [151]:
base = DecisionTreeClassifier()
grid = {
"max_depth" : [10, 100, 1000],
"criterion" : ["gini", "entropy"]
}
model = GridSearchCV(base, grid, cv=cv,n_jobs=4, verbose=100, return_train_score = True)

In [152]:
dtl_X = np.nan_to_num(X)
model = model.fit(dtl_X.astype(np.int8),y)

Fitting 1 folds for each of 6 candidates, totalling 6 fits
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
Pickling array (shape=(10238, 12), dtype=int8).
Pickling array (shape=(10238,), dtype=int32).
Pickling array (shape=(7166,), dtype=int32).
Pickling array (shape=(3072,), dtype=int32).
Pickling array (shape=(10238, 12), dtype=int8).
Pickling array (shape=(10238,), dtype=int32).
Pickling array (shape=(7166,), dtype=int32).
Pickling array (shape=(3072,), dtype=int32).
Pickling array (shape=(10238, 12), dtype=int8).
Pickling array (shape=(10238,), dtype=int32).
Pickling array (shape=(7166,), dtype=int32).
Pickling array (shape=(3072,), dtype=int32).
Pickling array (shape=(10238, 12), dtype=int8).
Pickling array (shape=(10238,), dtype=int32).
Pickling array (shape=(7166,), dtype=int32).
Pickling array (shape=(3072,), dtype=int32).
Pickling array (shape=(10238, 12), dtype=int8).
Pickling array (shape=(10238,), dtype=int32).
Pickling array (shape=(7166,), dtype

In [153]:
print(model.cv_results_)

{'mean_fit_time': array([0.04003072, 0.07152939, 0.06704116, 0.05053806, 0.09450006,
       0.09696388]), 'std_fit_time': array([0., 0., 0., 0., 0., 0.]), 'mean_score_time': array([0.00099969, 0.00147128, 0.00148726, 0.00099492, 0.00100017,
       0.00100183]), 'std_score_time': array([0., 0., 0., 0., 0., 0.]), 'param_criterion': masked_array(data=['gini', 'gini', 'gini', 'entropy', 'entropy',
                   'entropy'],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_max_depth': masked_array(data=[10, 100, 1000, 10, 100, 1000],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'criterion': 'gini', 'max_depth': 10}, {'criterion': 'gini', 'max_depth': 100}, {'criterion': 'gini', 'max_depth': 1000}, {'criterion': 'entropy', 'max_depth': 10}, {'criterion': 'entropy', 'max_depth': 100}, {'criterion': 'entropy', 'max_depth': 1000}], 'split0_test

In [154]:
pd.concat([pd.DataFrame(model.cv_results_["rank_test_score"], columns=["Rank"]), pd.DataFrame(model.cv_results_["params"]),pd.DataFrame(model.cv_results_["mean_test_score"], columns=["Accuracy"]),pd.DataFrame(model.cv_results_["mean_fit_time"], columns=["Fit Time"])],axis=1)

,Rank,criterion,max_depth,Accuracy,Fit Time
0,1,gini,10,0.540690,0.040031
1,6,gini,100,0.512044,0.071529
2,5,gini,1000,0.512695,0.067041
3,3,entropy,10,0.526042,0.050538
4,4,entropy,100,0.522461,0.094500
5,2,entropy,1000,0.529622,0.096964


In [155]:
pd.concat([pd.DataFrame(model.cv_results_["rank_test_score"], columns=["Rank"]), pd.DataFrame(model.cv_results_["params"]),pd.DataFrame(model.cv_results_["mean_test_score"], columns=["Accuracy"]),pd.DataFrame(model.cv_results_["mean_train_score"], columns=["Train Score"])],axis=1)

,Rank,criterion,max_depth,Accuracy,Train Score
0,1,gini,10,0.540690,0.683505
1,6,gini,100,0.512044,1.000000
2,5,gini,1000,0.512695,1.000000
3,3,entropy,10,0.526042,0.643316
4,4,entropy,100,0.522461,1.000000
5,2,entropy,1000,0.529622,1.000000


In [156]:
dump(model, 'local_dtl_{}_{}_cm.pkl'.format('70p', 64))

['local_dtl_70p_64_cm.pkl']

In [157]:
# Random Forest classifier
rf_base = xgb.XGBRFClassifier()
rf_grid = {
"max_depth" : [4, 6, 10],
"learning_rate" : [0.1, 0.3],
"n_estimators" : [1, 5, 10, 50]
}
rf_model = GridSearchCV(rf_base, rf_grid, cv=cv,n_jobs=4, verbose=100, return_train_score = True)

In [158]:
rf_model = rf_model.fit(X, y)

Fitting 1 folds for each of 24 candidates, totalling 24 fits
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
Pickling array (shape=(10238, 12), dtype=float64).
Pickling array (shape=(10238,), dtype=int32).
Pickling array (shape=(7166,), dtype=int32).
Pickling array (shape=(3072,), dtype=int32).
Pickling array (shape=(10238, 12), dtype=float64).
Pickling array (shape=(10238,), dtype=int32).
Pickling array (shape=(7166,), dtype=int32).
Pickling array (shape=(3072,), dtype=int32).
Pickling array (shape=(10238, 12), dtype=float64).
Pickling array (shape=(10238,), dtype=int32).
Pickling array (shape=(7166,), dtype=int32).
Pickling array (shape=(3072,), dtype=int32).
Pickling array (shape=(10238, 12), dtype=float64).
Pickling array (shape=(10238,), dtype=int32).
Pickling array (shape=(7166,), dtype=int32).
Pickling array (shape=(3072,), dtype=int32).
Pickling array (shape=(10238, 12), dtype=float64).
Pickling array (shape=(10238,), dtype=int32).
Pickling array (sha

In [159]:
print(rf_model.cv_results_)

{'mean_fit_time': array([0.01699615, 0.03599524, 0.06599641, 0.26050043, 0.02300024,
       0.05249858, 0.07799911, 0.48349833, 0.02600217, 0.09149981,
       0.14200091, 0.73699832, 0.02049851, 0.03999901, 0.06499696,
       0.35449958, 0.02349973, 0.08249974, 0.16050005, 0.3599987 ,
       0.03999996, 0.08349919, 0.1349988 , 0.47399926]), 'std_fit_time': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.]), 'mean_score_time': array([0.00399995, 0.00400209, 0.00400043, 0.00500107, 0.00400043,
       0.00299931, 0.00400233, 0.00749993, 0.00349784, 0.00349998,
       0.00399947, 0.00949955, 0.00349998, 0.00350142, 0.00300121,
       0.00550127, 0.00349951, 0.00500035, 0.00449944, 0.00700164,
       0.0040009 , 0.00500011, 0.00449944, 0.00749993]), 'std_score_time': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.]), 'param_learning_rate': masked_array(data=[0.1, 0.1, 0.1

In [160]:
pd.concat([pd.DataFrame(rf_model.cv_results_["rank_test_score"], columns=["Rank"]), pd.DataFrame(rf_model.cv_results_["params"]),pd.DataFrame(rf_model.cv_results_["mean_test_score"], columns=["Accuracy"]),pd.DataFrame(rf_model.cv_results_["mean_fit_time"], columns=["Fit Time"])],axis=1)

,Rank,learning_rate,max_depth,n_estimators,Accuracy,Fit Time
0,23,0.1,4,1,0.534505,0.016996
1,13,0.1,4,5,0.567708,0.035995
2,7,0.1,4,10,0.572917,0.065996
3,1,0.1,4,50,0.580078,0.260500
4,19,0.1,6,1,0.548828,0.023000
5,13,0.1,6,5,0.567708,0.052499
6,9,0.1,6,10,0.570964,0.077999
7,1,0.1,6,50,0.580078,0.483498
8,21,0.1,10,1,0.540365,0.026002
9,17,0.1,10,5,0.563477,0.091500


In [161]:
pd.concat([pd.DataFrame(rf_model.cv_results_["rank_test_score"], columns=["Rank"]), pd.DataFrame(rf_model.cv_results_["params"]),pd.DataFrame(rf_model.cv_results_["mean_test_score"], columns=["Accuracy"]),pd.DataFrame(rf_model.cv_results_["mean_train_score"], columns=["Train Score"])],axis=1)


,Rank,learning_rate,max_depth,n_estimators,Accuracy,Train Score
0,23,0.1,4,1,0.534505,0.572984
1,13,0.1,4,5,0.567708,0.591822
2,7,0.1,4,10,0.572917,0.592241
3,1,0.1,4,50,0.580078,0.598102
4,19,0.1,6,1,0.548828,0.588473
5,13,0.1,6,5,0.567708,0.623919
6,9,0.1,6,10,0.570964,0.638431
7,1,0.1,6,50,0.580078,0.644572
8,21,0.1,10,1,0.540365,0.631035
9,17,0.1,10,5,0.563477,0.746162


In [162]:
dump(rf_model, 'local_rf_{}_{}_cm.pkl'.format('70p', 64))

['local_rf_70p_64_cm.pkl']

In [163]:
xg_base = xgb.XGBClassifier()
xg_grid = {
"max_depth" : [4, 6, 10],
"learning_rate" : [0.1, 0.3],
"n_estimators" : [1, 5, 10, 50]
}
xg_model = GridSearchCV(xg_base, xg_grid, cv=cv, n_jobs=4, verbose=100, return_train_score = True)

In [164]:
xg_model = xg_model.fit(X, y)

Fitting 1 folds for each of 24 candidates, totalling 24 fits
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
Pickling array (shape=(10238, 12), dtype=float64).
Pickling array (shape=(10238,), dtype=int32).
Pickling array (shape=(7166,), dtype=int32).
Pickling array (shape=(3072,), dtype=int32).
Pickling array (shape=(10238, 12), dtype=float64).
Pickling array (shape=(10238,), dtype=int32).
Pickling array (shape=(7166,), dtype=int32).
Pickling array (shape=(3072,), dtype=int32).
Pickling array (shape=(10238, 12), dtype=float64).
Pickling array (shape=(10238,), dtype=int32).
Pickling array (shape=(7166,), dtype=int32).
Pickling array (shape=(3072,), dtype=int32).
Pickling array (shape=(10238, 12), dtype=float64).
Pickling array (shape=(10238,), dtype=int32).
Pickling array (shape=(7166,), dtype=int32).
Pickling array (shape=(3072,), dtype=int32).
Pickling array (shape=(10238, 12), dtype=float64).
Pickling array (shape=(10238,), dtype=int32).
Pickling array (sha

In [165]:
print(xg_model.cv_results_)

{'mean_fit_time': array([0.02099943, 0.04650021, 0.08100343, 0.34550047, 0.02450252,
       0.07201338, 0.13000131, 0.52150226, 0.03300095, 0.12899923,
       0.2385025 , 0.82899976, 0.02250051, 0.04699898, 0.08651376,
       0.34200001, 0.02649951, 0.06149983, 0.11249924, 0.48849916,
       0.02998972, 0.11750031, 0.17449927, 0.58299828]), 'std_fit_time': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.]), 'mean_score_time': array([0.00299931, 0.00399923, 0.00449824, 0.00600243, 0.00349903,
       0.00398636, 0.00349689, 0.00699949, 0.00349951, 0.00400257,
       0.00399899, 0.00600052, 0.00400043, 0.0040009 , 0.00248384,
       0.00500131, 0.0035007 , 0.00450158, 0.00450277, 0.00551081,
       0.00350046, 0.00499988, 0.00249958, 0.00600195]), 'std_score_time': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.]), 'param_learning_rate': masked_array(data=[0.1, 0.1, 0.1

In [167]:
pd.concat([pd.DataFrame(xg_model.cv_results_["rank_test_score"], columns=["Rank"]), pd.DataFrame(xg_model.cv_results_["params"]),pd.DataFrame(xg_model.cv_results_["mean_test_score"], columns=["Accuracy"]),pd.DataFrame(xg_model.cv_results_["mean_fit_time"], columns=["Fit Time"])],axis=1)




,Rank,learning_rate,max_depth,n_estimators,Accuracy,Fit Time
0,21,0.1,4,1,0.549805,0.020999
1,13,0.1,4,5,0.563802,0.046500
2,3,0.1,4,10,0.574870,0.081003
3,5,0.1,4,50,0.573893,0.345500
4,23,0.1,6,1,0.548503,0.024503
5,17,0.1,6,5,0.557943,0.072013
6,11,0.1,6,10,0.565104,0.130001
7,7,0.1,6,50,0.570638,0.521502
8,19,0.1,10,1,0.553711,0.033001
9,18,0.1,10,5,0.554036,0.128999


In [168]:
pd.concat([pd.DataFrame(xg_model.cv_results_["rank_test_score"], columns=["Rank"]), pd.DataFrame(xg_model.cv_results_["params"]),pd.DataFrame(xg_model.cv_results_["mean_test_score"], columns=["Accuracy"]),pd.DataFrame(xg_model.cv_results_["mean_train_score"], columns=["Train Score"])],axis=1)

,Rank,learning_rate,max_depth,n_estimators,Accuracy,Train Score
0,21,0.1,4,1,0.549805,0.583729
1,13,0.1,4,5,0.563802,0.591404
2,3,0.1,4,10,0.574870,0.605219
3,5,0.1,4,50,0.573893,0.640943
4,23,0.1,6,1,0.548503,0.605917
5,17,0.1,6,5,0.557943,0.631175
6,11,0.1,6,10,0.565104,0.655317
7,7,0.1,6,50,0.570638,0.735138
8,19,0.1,10,1,0.553711,0.692018
9,18,0.1,10,5,0.554036,0.763048


In [166]:
dump(xg_model, 'local_xg_{}_{}_cm.pkl'.format('70p', 64))

['local_xg_70p_64_cm.pkl']

In [169]:
all_features_train = pd.concat([glcm_data_train, color_moment_data_train],  axis=1, sort=False)
all_features_test = pd.concat([glcm_data_test, color_moment_data_test],  axis=1, sort=False)

In [170]:
# Eksperimen dengan seluruh data
test_fold = np.concatenate([
    # The training data.
    np.full(len(all_features_train), -1, dtype=np.int8),
    # The development data.
    np.zeros(len(all_features_test), dtype=np.int8)
])

cv = PredefinedSplit(test_fold)

X = np.concatenate([all_features_train, all_features_test])
y = np.concatenate([y_train, y_test])

In [171]:
base = DecisionTreeClassifier()
grid = {
"max_depth" : [10, 100, 1000],
"criterion" : ["gini", "entropy"]
}
model = GridSearchCV(base, grid, cv=cv,n_jobs=4, verbose=100, return_train_score = True)

In [172]:
dtl_X = np.nan_to_num(X)
model = model.fit(dtl_X.astype(np.int8),y)

Fitting 1 folds for each of 6 candidates, totalling 6 fits
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
Pickling array (shape=(10238, 18), dtype=int8).
Pickling array (shape=(10238,), dtype=int32).
Pickling array (shape=(7166,), dtype=int32).
Pickling array (shape=(3072,), dtype=int32).
Pickling array (shape=(10238, 18), dtype=int8).
Pickling array (shape=(10238,), dtype=int32).
Pickling array (shape=(7166,), dtype=int32).
Pickling array (shape=(3072,), dtype=int32).
Pickling array (shape=(10238, 18), dtype=int8).
Pickling array (shape=(10238,), dtype=int32).
Pickling array (shape=(7166,), dtype=int32).
Pickling array (shape=(3072,), dtype=int32).
Pickling array (shape=(10238, 18), dtype=int8).
Pickling array (shape=(10238,), dtype=int32).
Pickling array (shape=(7166,), dtype=int32).
Pickling array (shape=(3072,), dtype=int32).
Pickling array (shape=(10238, 18), dtype=int8).
Pickling array (shape=(10238,), dtype=int32).
Pickling array (shape=(7166,), dtype

In [173]:
print(model.cv_results_)

{'mean_fit_time': array([0.04944849, 0.07149673, 0.07653236, 0.06499624, 0.10202837,
       0.0969975 ]), 'std_fit_time': array([0., 0., 0., 0., 0., 0.]), 'mean_score_time': array([0.00099635, 0.00099969, 0.00100112, 0.00049996, 0.00097227,
       0.00102401]), 'std_score_time': array([0., 0., 0., 0., 0., 0.]), 'param_criterion': masked_array(data=['gini', 'gini', 'gini', 'entropy', 'entropy',
                   'entropy'],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_max_depth': masked_array(data=[10, 100, 1000, 10, 100, 1000],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'criterion': 'gini', 'max_depth': 10}, {'criterion': 'gini', 'max_depth': 100}, {'criterion': 'gini', 'max_depth': 1000}, {'criterion': 'entropy', 'max_depth': 10}, {'criterion': 'entropy', 'max_depth': 100}, {'criterion': 'entropy', 'max_depth': 1000}], 'split0_test

In [174]:
pd.concat([pd.DataFrame(model.cv_results_["rank_test_score"], columns=["Rank"]), pd.DataFrame(model.cv_results_["params"]),pd.DataFrame(model.cv_results_["mean_test_score"], columns=["Accuracy"]),pd.DataFrame(model.cv_results_["mean_fit_time"], columns=["Fit Time"])],axis=1)

,Rank,criterion,max_depth,Accuracy,Fit Time
0,2,gini,10,0.564453,0.049448
1,4,gini,100,0.534505,0.071497
2,6,gini,1000,0.528320,0.076532
3,1,entropy,10,0.568685,0.064996
4,5,entropy,100,0.530924,0.102028
5,3,entropy,1000,0.535482,0.096997


In [175]:
pd.concat([pd.DataFrame(model.cv_results_["rank_test_score"], columns=["Rank"]), pd.DataFrame(model.cv_results_["params"]),pd.DataFrame(model.cv_results_["mean_test_score"], columns=["Accuracy"]),pd.DataFrame(model.cv_results_["mean_train_score"], columns=["Train Score"])],axis=1)

,Rank,criterion,max_depth,Accuracy,Train Score
0,2,gini,10,0.564453,0.721881
1,4,gini,100,0.534505,1.000000
2,6,gini,1000,0.528320,1.000000
3,1,entropy,10,0.568685,0.701647
4,5,entropy,100,0.530924,1.000000
5,3,entropy,1000,0.535482,1.000000


In [176]:
dump(model, 'local_dtl_{}_{}_all.pkl'.format('70p', 64))

['local_dtl_70p_64_all.pkl']

In [177]:
# Random Forest classifier
rf_base = xgb.XGBRFClassifier()
rf_grid = {
"max_depth" : [4, 6, 10],
"learning_rate" : [0.1, 0.3],
"n_estimators" : [1, 5, 10, 50]
}
rf_model = GridSearchCV(rf_base, rf_grid, cv=cv,n_jobs=4, verbose=100, return_train_score = True)

In [178]:
rf_model = rf_model.fit(X, y)

Fitting 1 folds for each of 24 candidates, totalling 24 fits
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
Memmapping (shape=(10238, 18), dtype=float64) to new file C:\Users\glaci\AppData\Local\Temp\joblib_memmapping_folder_125832_5244594339\125832-1865479779912-91be64cf792f4ddb9788e0d4b13b2c18.pkl
Pickling array (shape=(10238,), dtype=int32).
Pickling array (shape=(7166,), dtype=int32).
Pickling array (shape=(3072,), dtype=int32).
Memmapping (shape=(10238, 18), dtype=float64) to old file C:\Users\glaci\AppData\Local\Temp\joblib_memmapping_folder_125832_5244594339\125832-1865479779912-91be64cf792f4ddb9788e0d4b13b2c18.pkl
Pickling array (shape=(10238,), dtype=int32).
Pickling array (shape=(7166,), dtype=int32).
Pickling array (shape=(3072,), dtype=int32).
Memmapping (shape=(10238, 18), dtype=float64) to old file C:\Users\glaci\AppData\Local\Temp\joblib_memmapping_folder_125832_5244594339\125832-1865479779912-91be64cf792f4ddb9788e0d4b13b2c18.pkl
Pickling arra

In [179]:
print(rf_model.cv_results_)

{'mean_fit_time': array([0.02299786, 0.05299997, 0.08899808, 0.35449696, 0.02849793,
       0.07150006, 0.12150145, 0.57100081, 0.04849935, 0.13200283,
       0.23350024, 0.92349839, 0.02799964, 0.05499768, 0.08350205,
       0.35649848, 0.03050065, 0.0830009 , 0.12549901, 0.53449821,
       0.03899932, 0.13400197, 0.19750094, 0.68149853]), 'std_fit_time': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.]), 'mean_score_time': array([0.00400043, 0.00499821, 0.00399995, 0.00550175, 0.00400114,
       0.00349927, 0.00449777, 0.00799918, 0.00300193, 0.00449705,
       0.00500035, 0.01349807, 0.00349998, 0.00500393, 0.00599718,
       0.00650048, 0.00350142, 0.00450087, 0.00450253, 0.00800014,
       0.00500202, 0.00649858, 0.0049994 , 0.00849938]), 'std_score_time': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.]), 'param_learning_rate': masked_array(data=[0.1, 0.1, 0.1

In [180]:
pd.concat([pd.DataFrame(rf_model.cv_results_["rank_test_score"], columns=["Rank"]), pd.DataFrame(rf_model.cv_results_["params"]),pd.DataFrame(rf_model.cv_results_["mean_test_score"], columns=["Accuracy"]),pd.DataFrame(rf_model.cv_results_["mean_fit_time"], columns=["Fit Time"])],axis=1)

,Rank,learning_rate,max_depth,n_estimators,Accuracy,Fit Time
0,17,0.1,4,1,0.578451,0.022998
1,11,0.1,4,5,0.585612,0.053000
2,11,0.1,4,10,0.585612,0.088998
3,9,0.1,4,50,0.589844,0.354497
4,21,0.1,6,1,0.561849,0.028498
5,3,0.1,6,5,0.598958,0.071500
6,5,0.1,6,10,0.598633,0.121501
7,1,0.1,6,50,0.599609,0.571001
8,23,0.1,10,1,0.557617,0.048499
9,19,0.1,10,5,0.573893,0.132003


In [181]:
pd.concat([pd.DataFrame(rf_model.cv_results_["rank_test_score"], columns=["Rank"]), pd.DataFrame(rf_model.cv_results_["params"]),pd.DataFrame(rf_model.cv_results_["mean_test_score"], columns=["Accuracy"]),pd.DataFrame(rf_model.cv_results_["mean_train_score"], columns=["Train Score"])],axis=1)

,Rank,learning_rate,max_depth,n_estimators,Accuracy,Train Score
0,17,0.1,4,1,0.578451,0.604661
1,11,0.1,4,5,0.585612,0.616941
2,11,0.1,4,10,0.585612,0.614987
3,9,0.1,4,50,0.589844,0.617360
4,21,0.1,6,1,0.561849,0.618616
5,3,0.1,6,5,0.598958,0.657550
6,5,0.1,6,10,0.598633,0.663690
7,1,0.1,6,50,0.599609,0.669411
8,23,0.1,10,1,0.557617,0.684203
9,19,0.1,10,5,0.573893,0.807284


In [182]:
dump(rf_model, 'local_rf_{}_{}_all.pkl'.format('70p', 64))

['local_rf_70p_64_all.pkl']

In [183]:
xg_base = xgb.XGBClassifier()
xg_grid = {
"max_depth" : [4, 6, 10],
"learning_rate" : [0.1, 0.3],
"n_estimators" : [1, 5, 10, 50]
}
xg_model = GridSearchCV(xg_base, xg_grid, cv=cv, n_jobs=4, verbose=100, return_train_score = True)

In [184]:
xg_model = xg_model.fit(X, y)

Fitting 1 folds for each of 24 candidates, totalling 24 fits
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
Memmapping (shape=(10238, 18), dtype=float64) to new file C:\Users\glaci\AppData\Local\Temp\joblib_memmapping_folder_125832_5244594339\125832-1865479779912-91be64cf792f4ddb9788e0d4b13b2c18.pkl
Pickling array (shape=(10238,), dtype=int32).
Pickling array (shape=(7166,), dtype=int32).
Pickling array (shape=(3072,), dtype=int32).
Memmapping (shape=(10238, 18), dtype=float64) to old file C:\Users\glaci\AppData\Local\Temp\joblib_memmapping_folder_125832_5244594339\125832-1865479779912-91be64cf792f4ddb9788e0d4b13b2c18.pkl
Pickling array (shape=(10238,), dtype=int32).
Pickling array (shape=(7166,), dtype=int32).
Pickling array (shape=(3072,), dtype=int32).
Memmapping (shape=(10238, 18), dtype=float64) to old file C:\Users\glaci\AppData\Local\Temp\joblib_memmapping_folder_125832_5244594339\125832-1865479779912-91be64cf792f4ddb9788e0d4b13b2c18.pkl
Pickling arra

In [185]:
print(xg_model.cv_results_)

{'mean_fit_time': array([0.03200173, 0.06349921, 0.10299873, 0.45699835, 0.03500032,
       0.08498931, 0.17899942, 0.81250119, 0.05099773, 0.16150284,
       0.36799836, 1.11749887, 0.03299761, 0.07050014, 0.12950015,
       0.48800254, 0.04750085, 0.11599946, 0.17749929, 0.65200162,
       0.05200076, 0.16349721, 0.241503  , 0.78000164]), 'std_fit_time': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.]), 'mean_score_time': array([0.00449777, 0.00400019, 0.00400066, 0.00600123, 0.00349832,
       0.00450015, 0.00450039, 0.00849962, 0.00349903, 0.00549722,
       0.00550079, 0.00699925, 0.00499988, 0.00450039, 0.00999999,
       0.00599718, 0.00450301, 0.00549865, 0.00549936, 0.00849938,
       0.00549865, 0.00450063, 0.00499845, 0.00600028]), 'std_score_time': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.]), 'param_learning_rate': masked_array(data=[0.1, 0.1, 0.1

In [187]:
pd.concat([pd.DataFrame(xg_model.cv_results_["rank_test_score"], columns=["Rank"]), pd.DataFrame(xg_model.cv_results_["params"]),pd.DataFrame(xg_model.cv_results_["mean_test_score"], columns=["Accuracy"]),pd.DataFrame(xg_model.cv_results_["mean_fit_time"], columns=["Fit Time"])],axis=1)

,Rank,learning_rate,max_depth,n_estimators,Accuracy,Fit Time
0,8,0.1,4,1,0.584310,0.032002
1,4,0.1,4,5,0.594076,0.063499
2,2,0.1,4,10,0.596354,0.102999
3,3,0.1,4,50,0.595052,0.456998
4,16,0.1,6,1,0.571940,0.035000
5,6,0.1,6,5,0.591146,0.084989
6,8,0.1,6,10,0.584310,0.178999
7,12,0.1,6,50,0.581055,0.812501
8,23,0.1,10,1,0.561849,0.050998
9,18,0.1,10,5,0.570964,0.161503


In [188]:
pd.concat([pd.DataFrame(xg_model.cv_results_["rank_test_score"], columns=["Rank"]), pd.DataFrame(xg_model.cv_results_["params"]),pd.DataFrame(xg_model.cv_results_["mean_test_score"], columns=["Accuracy"]),pd.DataFrame(xg_model.cv_results_["mean_train_score"], columns=["Train Score"])],axis=1)

,Rank,learning_rate,max_depth,n_estimators,Accuracy,Train Score
0,8,0.1,4,1,0.584310,0.601172
1,4,0.1,4,5,0.594076,0.623919
2,2,0.1,4,10,0.596354,0.626570
3,3,0.1,4,50,0.595052,0.675412
4,16,0.1,6,1,0.571940,0.639967
5,6,0.1,6,5,0.591146,0.667876
6,8,0.1,6,10,0.584310,0.687692
7,12,0.1,6,50,0.581055,0.758024
8,23,0.1,10,1,0.561849,0.721183
9,18,0.1,10,5,0.570964,0.832263


In [186]:
dump(xg_model, 'local_xg_{}_{}_all.pkl'.format('70p', 64))

['local_xg_70p_64_all.pkl']

More on: https://github.com/otakbeku/modified-FCOS